#NLP Final Project - Phase 2

##Installing and Importing Dependencies


1.   sentence transformers: This dependency is bieng used to get embeddings of sentences.
2.   sklearn: This library is bieng used to cluster and classify datas in dataset.
3.   Pandas: This library helps us to work with dataframes
4.   Numpy: For working with numbers and arrays



In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from operator import is_not
from functools import partial

##Loading Dataset

In [ ]:
all_players_dataset=pd.read_csv('/content/drive/MyDrive/IUST/1401_02_Spring/Natural_Language_Processing/Final_Project/all_players.csv')
all_players_dataset.loc[0]['rating1']

'True'

##Pre_Process Dataset

In [ ]:
df_cols=all_players_dataset.columns

sents_columns=[col if col[:4]=='sent' else None for col in df_cols]
sents_columns=list(filter(partial(is_not, None), sents_columns))
sents_columns_len=len(sents_columns)

players_columns=[col if col[:6]=='player' else None for col in df_cols]
players_columns=list(filter(partial(is_not, None), players_columns))
players_columns_len=len(players_columns)

ratings_columns=[col if col[:6]=='rating' else None for col in df_cols]
ratings_columns=list(filter(partial(is_not, None), ratings_columns))
ratings_columns_len=len(ratings_columns)

In [ ]:
players_df={
    'name':[],
    'sents':[],
    'rating':[]
}

for row in range(len(all_players_dataset)):
  for player_idx in range(players_columns_len):
    if(all_players_dataset.loc[row]['rating'+str(player_idx)] is not None):
      players_df['name'].append(all_players_dataset.loc[row]['player'+str(player_idx)])
      sents=[]
      for sent in range(sents_columns_len):
        if(all_players_dataset.loc[row]['player'+str(player_idx)] in str(all_players_dataset.loc[row]['sent'+str(sent)]).split()):
          sents.append(all_players_dataset.loc[row]['sent'+str(sent)])
      players_df['sents'].append(sents)
      if(all_players_dataset.loc[row]['rating'+str(player_idx)]=='True'):
        players_df['rating'].append(1)
      else:
        players_df['rating'].append(0)

In [ ]:
players_df=pd.DataFrame.from_dict(players_df)
players_df=players_df[players_df.name != '<PAD>']
players_df=players_df[players_df.astype(str)['sents'] != '[]']
players_df

,name,sents,rating
0,McCarthy,[City's sensational season appeared to be endi...,1
1,Yoshida,[City piled on the pressure in the second half...,1
19,Fernandinho,[After Fernandinho cleared Dusan Tadic's effor...,1
24,Sterling,[After no attempts on target from City in the ...,0
25,Jesus,[City's sensational season appeared to be endi...,1
...,...,...,...
67843,Johnston,[Rosenborg goalkeeper Andre Hansen denied Tom ...,0
67844,Hansen,[Rosenborg goalkeeper Andre Hansen denied Tom ...,0
67845,Meling,"[On 35 minutes, Kieran Tierney broke clear on ...",0
67848,Hedenstad,[He constantly caused Yann Erik de-Lanlay and ...,0


##Feature Extraction

In [ ]:
total_sents=[]
for sents in players_df['sents']:
  for sent in sents:
    if(sent!='<PAD>'):
      total_sents.append(sent)

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
total_sents_embedding=model.encode(total_sents)

In [ ]:
kmeans = KMeans(n_clusters=50, random_state=0, n_init="auto")
kmeans.fit(total_sents_embedding)

KMeans(n_clusters=50, n_init='auto', random_state=0)

In [ ]:
for i in range(kmeans.n_clusters):
  players_df.insert(2,str(i),[0]*len(players_df))

In [ ]:
players_df

,name,sents,49,48,47,46,45,44,43,42,...,8,7,6,5,4,3,2,1,0,rating
0,McCarthy,[City's sensational season appeared to be endi...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Yoshida,[City piled on the pressure in the second half...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19,Fernandinho,[After Fernandinho cleared Dusan Tadic's effor...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24,Sterling,[After no attempts on target from City in the ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,Jesus,[City's sensational season appeared to be endi...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67843,Johnston,[Rosenborg goalkeeper Andre Hansen denied Tom ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67844,Hansen,[Rosenborg goalkeeper Andre Hansen denied Tom ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67845,Meling,"[On 35 minutes, Kieran Tierney broke clear on ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67848,Hedenstad,[He constantly caused Yann Erik de-Lanlay and ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
players_df.at[0,'1']=0
players_df

,name,sents,49,48,47,46,45,44,43,42,...,8,7,6,5,4,3,2,1,0,rating
0,McCarthy,[City's sensational season appeared to be endi...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Yoshida,[City piled on the pressure in the second half...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19,Fernandinho,[After Fernandinho cleared Dusan Tadic's effor...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24,Sterling,[After no attempts on target from City in the ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,Jesus,[City's sensational season appeared to be endi...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67843,Johnston,[Rosenborg goalkeeper Andre Hansen denied Tom ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67844,Hansen,[Rosenborg goalkeeper Andre Hansen denied Tom ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67845,Meling,"[On 35 minutes, Kieran Tierney broke clear on ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67848,Hedenstad,[He constantly caused Yann Erik de-Lanlay and ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
players_df.reset_index(drop=True,inplace=True)
players_df

,name,sents,49,48,47,46,45,44,43,42,...,8,7,6,5,4,3,2,1,0,rating
0,McCarthy,[City's sensational season appeared to be endi...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Yoshida,[City piled on the pressure in the second half...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Fernandinho,[After Fernandinho cleared Dusan Tadic's effor...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Sterling,[After no attempts on target from City in the ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Jesus,[City's sensational season appeared to be endi...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20252,Johnston,[Rosenborg goalkeeper Andre Hansen denied Tom ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20253,Hansen,[Rosenborg goalkeeper Andre Hansen denied Tom ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20254,Meling,"[On 35 minutes, Kieran Tierney broke clear on ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20255,Hedenstad,[He constantly caused Yann Erik de-Lanlay and ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for row in range(len(players_df)):
  for sent in players_df.loc[row]['sents']:
    sent_cluster=str(kmeans.predict(model.encode([sent]))[0])
    players_df.at[row,sent_cluster]=players_df.loc[row][sent_cluster]+1

players_df

,name,sents,49,48,47,46,45,44,43,42,...,8,7,6,5,4,3,2,1,0,rating
0,McCarthy,[City's sensational season appeared to be endi...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Yoshida,[City piled on the pressure in the second half...,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,Fernandinho,[After Fernandinho cleared Dusan Tadic's effor...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Sterling,[After no attempts on target from City in the ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Jesus,[City's sensational season appeared to be endi...,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20252,Johnston,[Rosenborg goalkeeper Andre Hansen denied Tom ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20253,Hansen,[Rosenborg goalkeeper Andre Hansen denied Tom ...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
20254,Meling,"[On 35 minutes, Kieran Tierney broke clear on ...",0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
20255,Hedenstad,[He constantly caused Yann Erik de-Lanlay and ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
players_df.to_csv('/content/drive/MyDrive/IUST/1401_02_Spring/Natural_Language_Processing/Final_Project/features_extracted_players.csv')

##Players Classification

In [ ]:
players_df=pd.read_csv('/content/drive/MyDrive/IUST/1401_02_Spring/Natural_Language_Processing/Final_Project/features_extracted_players.csv')

In [ ]:
players_df

,Unnamed: 0,name,sents,49,48,47,46,45,44,43,...,8,7,6,5,4,3,2,1,0,rating
0,0,McCarthy,"[""City's sensational season appeared to be end...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,Yoshida,"[""City piled on the pressure in the second hal...",0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,2,Fernandinho,"[""After Fernandinho cleared Dusan Tadic's effo...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,Sterling,"[""After no attempts on target from City in the...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Jesus,"[""City's sensational season appeared to be end...",0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20252,20252,Johnston,"[""Rosenborg goalkeeper Andre Hansen denied Tom...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20253,20253,Hansen,"[""Rosenborg goalkeeper Andre Hansen denied Tom...",0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
20254,20254,Meling,"['On 35 minutes, Kieran Tierney broke clear on...",0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
20255,20255,Hedenstad,['He constantly caused Yann Erik de-Lanlay and...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def rate_to_grade(rate):
  if rate==True:
    return 1
  return 0

In [ ]:
x_train=players_df.iloc[:-100,3:-3]
y_train=players_df.iloc[:-100,-2]
x_test=players_df.iloc[-100:,3:-3]
y_test=players_df.iloc[-100:,-2]

In [ ]:
classification_model = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(x_train, y_train)

In [ ]:
classification_model.predict(x_test)
round(classification_model.score(x_test,y_test), 4)

1.0

##Finalizing Model

In [1]:
def SoccerReportPredictor(players,report):
  global model
  global kmeans
  global classification_model

  player_sents={}

  for player in players:
    player_sents[player]=[]
    for sent in report.split('.'):
      if player in sent.split():
        player_sents[player].append(kmeans.predict(model.encode(sent)))
  player_features={}
  for player in  player_sents:
     features=[0]*50
     for sent in player_sents[player]:
        features[sent]+=1
     player_features[player]=features

  player_features=pd.DataFrame.from_dict(player_features)
  return classification_model.predict(player_features[1:])

